In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 46.1 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.12-py3-none-any.whl size=102986 sha256=2d0d13c40c59323892c16a14d5b6955fe16dc383ad0ee4195c7be7547a42717f
  Stored in directory: /tmp/pip-ephem-wheel-cache-_04dvqiv/wheels/00/74/7d/52c235c074504ceced3dee430f10873c95d8a9e81263ece28e
Successfully built kaggle

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/opt/app-root/src/openshift-ai/cat-dog-predict/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/opt/app-root/src/openshift-ai/cat-dog-predict')
zip_ref.close()

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

2024-05-07 02:14:04.008060: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 02:14:04.008137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 02:14:04.045537: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-07 02:14:04.116819: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 02:14:06.524115: W tensorflow/compiler/tf2

In [2]:
! pip install mlflow
import mlflow

mlflow.set_tracking_uri('http://mlflow-server.mlflow.svc.cluster.local:8080')
mlflow.set_experiment("cat-dog")

mlflow.tensorflow.autolog()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 173.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 195.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 259.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 178.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 103.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 227.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 190.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 155.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 192.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 247.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 150.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 614.3/614.3 kB 256.0 MB/s eta 0:00:00
  Attempti

In [3]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/opt/app-root/src/openshift-ai/cat-dog-predict/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/opt/app-root/src/openshift-ai/cat-dog-predict/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [4]:
# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [5]:
# create CNN model

model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 254, 254, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_1 (Bat  (None, 125, 125, 64)      256       
 chNormalization)                                                
                                                        

In [7]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [8]:
history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

2024/05/07 02:14:25 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f734f66efd7a4d65b2c0e97e140cd888', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow
2024/05/07 02:14:32 INFO mlflow.types.utils: MLflow 2.9.0 introduces model signature with new data types for lists and dictionaries. For input such as Dict[str, Union[scalars, List, Dict]], we infer dictionary values types as `List -> Array` and `Dict -> Object`. 
2024/05/07 02:14:34 INFO mlflow.types.utils: MLflow 2.9.0 introduces model signature with new data types for lists and dictionaries. For input such as Dict[str, Union[scalars, List, Dict]], we infer dictionary values types as `List -> Array` and `Dict -> Object`. 


Epoch 1/10
625/625 [==============================] - ETA: 0s - loss: 1.3504 - accuracy: 0.5817

/opt/app-root/lib64/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


625/625 [==============================] - 468s 746ms/step - loss: 1.3504 - accuracy: 0.5817 - val_loss: 0.6381 - val_accuracy: 0.6194
Epoch 2/10
625/625 [==============================] - ETA: 0s - loss: 0.5649 - accuracy: 0.7113

2024/05/07 02:32:33 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: API request to http://mlflow-server.mlflow.svc.cluster.local:8080/api/2.0/mlflow-artifacts/artifacts/3/f734f66efd7a4d65b2c0e97e140cd888/artifacts/checkpoints/latest_checkpoint.h5 failed with exception HTTPConnectionPool(host='mlflow-server.mlflow.svc.cluster.local', port=8080): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/3/f734f66efd7a4d65b2c0e97e140cd888/artifacts/checkpoints/latest_checkpoint.h5 (Caused by ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))


625/625 [==============================] - 574s 919ms/step - loss: 0.5649 - accuracy: 0.7113 - val_loss: 0.5736 - val_accuracy: 0.7120
Epoch 3/10
625/625 [==============================] - 547s 875ms/step - loss: 0.4794 - accuracy: 0.7738 - val_loss: 0.4783 - val_accuracy: 0.7684
Epoch 4/10
625/625 [==============================] - 422s 675ms/step - loss: 0.4186 - accuracy: 0.8098 - val_loss: 0.4960 - val_accuracy: 0.7602
Epoch 5/10
625/625 [==============================] - 423s 677ms/step - loss: 0.3564 - accuracy: 0.8414 - val_loss: 0.6457 - val_accuracy: 0.6738
Epoch 6/10
625/625 [==============================] - 426s 681ms/step - loss: 0.3007 - accuracy: 0.8720 - val_loss: 0.7777 - val_accuracy: 0.7244
Epoch 7/10
625/625 [==============================] - 447s 716ms/step - loss: 0.2286 - accuracy: 0.9061 - val_loss: 0.6113 - val_accuracy: 0.7854
Epoch 8/10
625/625 [==============================] - ETA: 0s - loss: 0.1661 - accuracy: 0.9356

2024/05/07 03:19:36 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: API request to http://mlflow-server.mlflow.svc.cluster.local:8080/api/2.0/mlflow-artifacts/artifacts/3/f734f66efd7a4d65b2c0e97e140cd888/artifacts/checkpoints/latest_checkpoint.h5 failed with exception HTTPConnectionPool(host='mlflow-server.mlflow.svc.cluster.local', port=8080): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/3/f734f66efd7a4d65b2c0e97e140cd888/artifacts/checkpoints/latest_checkpoint.h5 (Caused by ResponseError('too many 500 error responses'))


625/625 [==============================] - 537s 860ms/step - loss: 0.1661 - accuracy: 0.9356 - val_loss: 0.5634 - val_accuracy: 0.8156
Epoch 9/10
625/625 [==============================] - 403s 644ms/step - loss: 0.1220 - accuracy: 0.9541 - val_loss: 2.3230 - val_accuracy: 0.6662
Epoch 10/10
1/1 [==============================] - 0s 141ms/step
INFO:tensorflow:Assets written to: /tmp/tmpdb23otrd/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpdb23otrd/model/data/model/assets
2024/05/07 03:36:43 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: API request to http://mlflow-server.mlflow.svc.cluster.local:8080/api/2.0/mlflow-artifacts/artifacts/3/f734f66efd7a4d65b2c0e97e140cd888/artifacts/model/data/model/variables/variables.data-00000-of-00001 failed with exception HTTPConnectionPool(host='mlflow-server.mlflow.svc.cluster.local', port=8080): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/3/f734f66efd7a4d65b2c0e97e140cd888/artifacts/model/data/model/variables/variables.data-00000-of-00001 (Caused by ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))


In [9]:
import mlflow

with mlflow.start_run():
  mlflow.log_param("model_type", "keras")  # Log additional parameters
  mlflow.keras.log_model(model, "model")

2024/05/07 03:36:43 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmpl2nbnkl8/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpl2nbnkl8/model/data/model/assets


In [ ]:
model.save('my_dog_cat_model.keras')

In [11]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "http://mlflow-server.mlflow.svc.cluster.local:8080"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.search_experiments()



[<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1714959733854, experiment_id='3', last_update_time=1714959733854, lifecycle_stage='active', name='cat-dog', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1714651176130, experiment_id='0', last_update_time=1714651176130, lifecycle_stage='active', name='Default', tags={}>]

In [12]:
# Interacting with the Model Registry
run_id = "b74ab30d6fd74c16a2ac8e0ee3689ace"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="cat_dogs")

Successfully registered model 'cat_dogs2'.
2024/05/07 03:42:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: cat_dogs2, version 1
Created version '1' of model 'cat_dogs2'.


<ModelVersion: aliases=[], creation_timestamp=1715053371228, current_stage='None', description='', last_updated_timestamp=1715053371228, name='cat_dogs2', run_id='b74ab30d6fd74c16a2ac8e0ee3689ace', run_link='', source='mlflow-artifacts:/3/b74ab30d6fd74c16a2ac8e0ee3689ace/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [13]:
model_name = "cat_dogs"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


/tmp/ipykernel_121/1222298901.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.1/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [14]:
model_version = 1
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=True
)

/tmp/ipykernel_121/3558476218.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.1/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1715053371228, current_stage='Production', description='', last_updated_timestamp=1715053503517, name='cat_dogs2', run_id='b74ab30d6fd74c16a2ac8e0ee3689ace', run_link='', source='mlflow-artifacts:/3/b74ab30d6fd74c16a2ac8e0ee3689ace/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [15]:
model_name = "cat_dogs"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Production


/tmp/ipykernel_121/1222298901.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.1/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)
